## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-05-01-13-37 +0000,bbc,US Supreme Court allows Texas to use redrawn v...,https://www.bbc.com/news/articles/cvg1v21p3ydo...
1,2025-12-05-01-13-09 +0000,bbc,Police recover $19k Fabergé egg swallowed by N...,https://www.bbc.com/news/articles/cj4qedglzpwo...
2,2025-12-05-01-00-11 +0000,nypost,‘Special place in hell’ for man accused of bru...,https://nypost.com/2025/12/04/us-news/special-...
3,2025-12-05-01-00-00 +0000,wsj,Tradition is everything in the wealthy Nashvil...,https://www.wsj.com/real-estate/luxury-homes/b...
4,2025-12-05-00-59-09 +0000,nyt,Trump Trumpets Peace in Africa Amid War on Dru...,https://www.nytimes.com/2025/12/04/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,53
180,new,20
127,he,15
186,ukraine,14
693,putin,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
58,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,122
167,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...,118
319,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,110
136,2025-12-04-21-10-58 +0000,nypost,Venezuelan dictator Maduro says his phone call...,https://nypost.com/2025/12/04/world-news/madur...,96
20,2025-12-05-00-21-26 +0000,nypost,Trump sees peace ‘eventually’ in Ukraine after...,https://nypost.com/2025/12/04/world-news/trump...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
58,122,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...
167,65,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...
99,61,2025-12-04-22-20-21 +0000,bbc,Putin says Russia will take Donbas by force or...,https://www.bbc.com/news/articles/cgexgwr1wrro...
19,57,2025-12-05-00-22-00 +0000,wsj,"In a briefing for lawmakers, Navy Adm. Frank “...",https://www.wsj.com/politics/national-security...
250,50,2025-12-04-14-37-28 +0000,nypost,"Netanyahu taunts Zohran Mamdani, saying he’ll ...",https://nypost.com/2025/12/04/us-news/benjamin...
246,48,2025-12-04-14-52-31 +0000,bbc,'Like first aid for manuscripts': Recovery wor...,https://www.bbc.com/news/articles/c157n1g977go...
48,43,2025-12-04-23-42-38 +0000,latimes,"Supreme Court rules for Texas Republicans, all...",https://www.latimes.com/politics/story/2025-12...
64,37,2025-12-04-23-11-43 +0000,nypost,Federal grand jury declines to indict NY AG Le...,https://nypost.com/2025/12/04/us-news/federal-...
277,34,2025-12-04-12-30-00 +0000,latimes,California hammered as national job cuts jump ...,https://www.latimes.com/business/story/2025-12...
319,33,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
